Importa bibliotecas, **frameworks** e arquivos .py<br>
keras: rede neural<br>
numpy: estrutura de dados<br>
pandas: estrutura de dados<br>
sklearn: normalização de dados


In [4]:
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras import optimizers
from keras.models import load_model
from keras.models import model_from_json

np.random.seed(7)

%run Python/simulation.py
%run Python/dataNumpy.py
import sys

from sklearn.preprocessing import Normalizer


In [5]:
headerInput = ['Hora', 'Temperature', 'Wind_Speed', 'Wind_Direction', 'Humidity', 'Air_Density', 'Rain_Status', 'Jardim_PMV','Jardim_Temp','Sala_PMV','Sala_Temp','Mezanino_PMV','Mezanino_Temp']
headerOutput = ['PortaSalaNorte','JanelaSalaNorte','PortaSalaLeste','JanelaSalaSul','JanelaSalaOeste','PortaCozinhaLeste','JanelaCozinhaSul','JanelaMezaninoNorte','JanelaMezaninoSul','JanelaMezaninoOeste']

print(len(headerInput))
print(len(headerOutput))

13
10


In [24]:
x = pd.read_csv("Selected/v02_input.csv")

xx = x[headerInput]

scaler_in = Normalizer().fit(xx)
xxx = scaler_in.transform(xx)

Função que permite alterar a PORTA da conexão TCP entre o energy plus, matlab e bcvtb

In [25]:
def changePort(PORT):
    base = open('simulateAndExit_base.m', 'r')
    f = open('simulateAndExit.m', 'w')
    lines = base.readlines()
    lines[37] = str(lines[37]).replace('50001', str(PORT))
    with open('simulateAndExit.m', 'w') as file_handler:
        for item in lines:
            file_handler.write("{}".format(item))
            
    base.close()
    f.close()

Modelo de rede neural testada

In [26]:
# load json and create model
json_file = open('Output_RN/v01_RN_t03_json.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("Output_RN/v01_RN_t03_model.h5")
print("Loaded model from disk")

sgd = optimizers.SGD(lr=0.1, decay=1e-5, momentum=0.3, nesterov=True)
loaded_model.compile(optimizer=sgd, loss='mean_squared_error', metrics=['mae', 'acc'])
loaded_model.summary()

print "Compilado"

Loaded model from disk
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 40)                560       
_________________________________________________________________
dense_8 (Dense)              (None, 40)                1640      
_________________________________________________________________
dense_9 (Dense)              (None, 40)                1640      
_________________________________________________________________
dense_10 (Dense)             (None, 40)                1640      
_________________________________________________________________
dense_11 (Dense)             (None, 40)                1640      
_________________________________________________________________
dense_12 (Dense)             (None, 10)                410       
Total params: 7,530
Trainable params: 7,530
Non-trainable params: 0
___________________________________________________

Regristra a rede neural, altera a porta da conexão TCP 

In [27]:
ID = "007"
PORT = int("50"+ID)
changePort(PORT)

In [28]:
## SIMULACAO
Simulate = Connection(PORT)
connec = Simulate.start()

NP_data = np.zeros((1, 23))
NP_saida = np.zeros((1, 10))

while True:
    data = connec[0].recv(1024)
    #print ("raw data:")
    #print (data)
    
    if not data:
        break

    #dados direto do matlab em string
    fromMatlab = data.decode("utf-8")

    #lista com
    dataAndTime = fromMatlab.split(",")

    weatherInText = dataAndTime[0].replace("[", "")
    weatherInText = weatherInText.replace("]", "")

    timeInfo = myTime.getTimeInfo(int(dataAndTime[1]))


    #Gera uma lista com os dados climatologicos
    weather = []
    for x in weatherInText.split(" "):
        weather.append(float(x))

    ##        Hora, TempExterna,
    """
    'Hora',
    'TempExterna', 'ArVeloc', 'ArDirecao', 'Umidade', 'ArDensidade', 'StatusChuva',
    'PMV_JardimInverno', 'Temp_JardimInverno',
    'PMV_SalaCozinha', 'Temp_SalaCozinha',
    'PMV_Mezanino', 'Temp_Mezanino'
    """
    
    """
    print "timeInfo:"
    print timeInfo
    
    print "weather: " + str(len(weather))
    print weather[19]
    """
    
    inputRN = pd.DataFrame([[timeInfo[2], weather[0], weather[1], weather[2], weather[3], weather[7], weather[8],
                weather[19], weather[20], weather[23], weather[24], weather[25], weather[26]]], columns=headerInput)
    
    inputRN_np = np.array(inputRN)
       
    np_inputRN = scaler_in.transform(inputRN_np)
    
    #print "np_inputRN:"
    #print np_inputRN
   
    #np_outputRN = model.predict_on_batch(np_inputRN)
    np_outputRN = np.around(loaded_model.predict(np_inputRN, verbose=0), 2)
    
    #np_outputRN = np_outputRN, 2) #ARREDONDA
    
    print np_outputRN
    
    np_timeInfo = np.array([timeInfo[2]])
    np_weather = np.array([weather])

    sendList = np_outputRN.tolist()[0]

    np_save = np.column_stack((inputRN_np, np_outputRN))
    

    
    NP_data = np.row_stack((NP_data, np_save))
    NP_saida = np.row_stack((NP_saida, np_outputRN))
    
    #l = pd.Series(np_save[0,:], index = Input + Output)

    
    sendToMatlab = ', '.join(str(float("%.2f" % e)) for e in sendList )

    #print("OUTPUT DATA:")
    #print (sendToMatlab)
    connec[0].sendall(str.encode(sendToMatlab))

   


Simulate.close()

arquivo = pd.DataFrame(NP_data)
arquivo.columns=headerInput+headerOutput
arquivo = arquivo[2:]
arquivo = arquivo[:-1]
arquivo.to_csv("Output_RN/v01_RN_"+ "t03" +".csv", index=False)


saida = pd.DataFrame(NP_saida)
saida.columns=headerOutput
saida = saida[2:]
saida = saida[:-1]
saida.to_csv("Output_RN/v01_RN_"+ "t03" +"_saida.csv", index=False)



print ("FIM!")


PYTHON || simulacao.py: Iniciando simulacao
PYTHON || simulacao.py: Esperando inicializacao do BCVTB
PYTHON || simulation.py: Recebendo dados
[[ 0.01        0.40000001  0.40000001  0.41        0.41999999  0.38999999
   0.37        0.38        0.38        0.41      ]]
[[ 0.03  0.08  0.09  0.08  0.09  0.09  0.09  0.08  0.08  0.08]]
[[ 0.03  0.07  0.08  0.07  0.08  0.08  0.08  0.07  0.07  0.07]]
[[ 0.04  0.06  0.06  0.06  0.07  0.06  0.06  0.06  0.06  0.06]]
[[ 0.04  0.06  0.06  0.05  0.07  0.06  0.06  0.06  0.05  0.05]]
[[ 0.04  0.06  0.06  0.05  0.06  0.06  0.06  0.06  0.05  0.05]]
[[ 0.04  0.06  0.06  0.05  0.06  0.06  0.06  0.06  0.05  0.05]]
[[ 0.04  0.06  0.06  0.05  0.07  0.06  0.06  0.06  0.06  0.06]]
[[ 0.03  0.07  0.07  0.07  0.08  0.07  0.07  0.07  0.07  0.07]]
[[ 0.01        0.44999999  0.44999999  0.44999999  0.47        0.44        0.41
   0.41999999  0.43000001  0.46000001]]
[[ 0.01        0.46000001  0.46000001  0.46000001  0.47999999  0.44999999
   0.41999999  0.43000001 

[[ 0.01        0.31999999  0.33000001  0.33000001  0.34        0.31999999
   0.30000001  0.31        0.31        0.34      ]]
[[ 0.01        0.31        0.31        0.31        0.31999999  0.30000001
   0.28999999  0.28999999  0.28999999  0.31999999]]
[[ 0.01        0.30000001  0.31        0.31        0.31999999  0.30000001
   0.28        0.28999999  0.28999999  0.31      ]]
[[ 0.01        0.23999999  0.25        0.25        0.25999999  0.25        0.23
   0.23999999  0.23999999  0.25      ]]
[[ 0.01        0.23999999  0.25        0.25        0.25999999  0.25        0.23
   0.23999999  0.23999999  0.25      ]]
[[ 0.01        0.23        0.23999999  0.23999999  0.25        0.23        0.22
   0.22        0.23        0.23999999]]
[[ 0.01        0.22        0.23        0.23        0.23999999  0.23        0.22
   0.22        0.22        0.23      ]]
[[ 0.01        0.23        0.23        0.23        0.23999999  0.23        0.22
   0.22        0.22        0.23999999]]
[[ 0.01        0.23   

[[ 0.01        0.49000001  0.49000001  0.49000001  0.50999999  0.47999999
   0.44999999  0.46000001  0.47        0.5       ]]
[[ 0.01        0.49000001  0.47999999  0.49000001  0.50999999  0.47
   0.44999999  0.46000001  0.47        0.5       ]]
[[ 0.01        0.49000001  0.47999999  0.49000001  0.5         0.47
   0.44999999  0.46000001  0.47        0.5       ]]
[[ 0.01        0.47999999  0.47999999  0.49000001  0.5         0.47        0.44
   0.46000001  0.46000001  0.49000001]]
[[ 0.01        0.47999999  0.47999999  0.49000001  0.5         0.47        0.44
   0.46000001  0.46000001  0.49000001]]
[[ 0.01        0.47999999  0.47999999  0.49000001  0.5         0.47        0.44
   0.46000001  0.46000001  0.49000001]]
[[ 0.01        0.47999999  0.47999999  0.49000001  0.5         0.47        0.44
   0.46000001  0.46000001  0.49000001]]
[[ 0.01        0.47        0.47        0.47999999  0.49000001  0.46000001
   0.43000001  0.44999999  0.44999999  0.47999999]]
[[ 0.01        0.47        0

[[ 0.01        0.44        0.44        0.44999999  0.46000001  0.43000001
   0.40000001  0.41999999  0.41999999  0.44999999]]
[[ 0.01        0.41999999  0.41999999  0.43000001  0.44        0.41        0.38
   0.40000001  0.40000001  0.43000001]]
[[ 0.01        0.41        0.41        0.41999999  0.43000001  0.40000001
   0.38        0.38999999  0.38999999  0.41999999]]
[[ 0.01        0.41        0.41        0.41        0.43000001  0.40000001
   0.37        0.38        0.38999999  0.41999999]]
[[ 0.01        0.41        0.41        0.41        0.43000001  0.40000001
   0.37        0.38        0.38999999  0.41999999]]
[[ 0.01        0.40000001  0.40000001  0.41        0.41999999  0.38999999
   0.37        0.38        0.38        0.41      ]]
[[ 0.01        0.47999999  0.47999999  0.49000001  0.5         0.47        0.44
   0.46000001  0.46000001  0.49000001]]
[[ 0.01        0.49000001  0.47999999  0.49000001  0.50999999  0.47
   0.44999999  0.46000001  0.47        0.5       ]]
[[ 0.01   

[[ 0.01        0.47        0.47        0.47        0.47999999  0.44999999
   0.43000001  0.44        0.44999999  0.47999999]]
[[ 0.01        0.47999999  0.47999999  0.49000001  0.5         0.47        0.44
   0.46000001  0.46000001  0.49000001]]
[[ 0.01        0.5         0.5         0.5         0.51999998  0.49000001
   0.46000001  0.47        0.47999999  0.50999999]]
[[ 0.01        0.47        0.47        0.47999999  0.49000001  0.46000001
   0.43000001  0.44999999  0.44999999  0.47999999]]
[[ 0.01        0.44999999  0.44999999  0.46000001  0.47        0.44
   0.41999999  0.43000001  0.44        0.47      ]]
[[ 0.01        0.44        0.44        0.44        0.46000001  0.43000001
   0.40000001  0.41        0.41999999  0.44999999]]
[[ 0.01        0.43000001  0.43000001  0.44        0.44999999  0.41999999
   0.40000001  0.41        0.41        0.44      ]]
[[ 0.01        0.41999999  0.41999999  0.43000001  0.44        0.41
   0.38999999  0.40000001  0.40000001  0.43000001]]
[[ 0.01   

...

[[ 0.03  0.07  0.07  0.06  0.08  0.07  0.07  0.07  0.07  0.07]]
[[ 0.03  0.07  0.08  0.07  0.08  0.08  0.07  0.07  0.07  0.07]]
[[ 0.03  0.07  0.08  0.07  0.08  0.08  0.08  0.07  0.07  0.07]]
[[ 0.03  0.07  0.08  0.07  0.08  0.08  0.07  0.07  0.07  0.07]]
[[ 0.03  0.07  0.07  0.07  0.08  0.07  0.07  0.07  0.07  0.07]]
[[ 0.03  0.07  0.07  0.07  0.08  0.07  0.07  0.07  0.07  0.07]]
[[ 0.04  0.07  0.07  0.06  0.08  0.07  0.07  0.07  0.06  0.06]]
[[ 0.04  0.07  0.07  0.06  0.08  0.07  0.07  0.07  0.07  0.07]]
[[ 0.01        0.36000001  0.36000001  0.36000001  0.38        0.34999999
   0.33000001  0.34        0.34        0.37      ]]
[[ 0.01        0.31        0.31        0.31999999  0.33000001  0.31
   0.28999999  0.30000001  0.30000001  0.31999999]]
[[ 0.01        0.34999999  0.34999999  0.34999999  0.37        0.34
   0.31999999  0.33000001  0.33000001  0.36000001]]
[[ 0.01        0.31        0.31999999  0.31999999  0.33000001  0.31
   0.28999999  0.30000001  0.30000001  0.33000001]]
[[

[[ 0.01        0.5         0.5         0.50999999  0.51999998  0.49000001
   0.46000001  0.47999999  0.47999999  0.50999999]]
[[ 0.01        0.49000001  0.49000001  0.5         0.50999999  0.47999999
   0.44999999  0.47        0.47        0.5       ]]
[[ 0.01        0.5         0.5         0.5         0.51999998  0.47999999
   0.46000001  0.47        0.47999999  0.50999999]]
[[ 0.01        0.49000001  0.49000001  0.5         0.50999999  0.47999999
   0.46000001  0.47        0.47999999  0.50999999]]
[[ 0.01        0.49000001  0.49000001  0.5         0.50999999  0.47999999
   0.44999999  0.47        0.47        0.5       ]]
[[ 0.01        0.47999999  0.47999999  0.49000001  0.5         0.47
   0.44999999  0.46000001  0.46000001  0.5       ]]
[[ 0.01        0.49000001  0.49000001  0.5         0.50999999  0.47999999
   0.44999999  0.47        0.47        0.5       ]]
[[ 0.01        0.49000001  0.49000001  0.5         0.50999999  0.47999999
   0.44999999  0.47        0.47        0.5       ]

[[ 0.01        0.50999999  0.5         0.50999999  0.52999997  0.49000001
   0.47        0.47999999  0.49000001  0.51999998]]
[[ 0.01        0.49000001  0.49000001  0.5         0.50999999  0.47999999
   0.44999999  0.47        0.47        0.5       ]]
[[ 0.01        0.47999999  0.47999999  0.49000001  0.5         0.47        0.44
   0.46000001  0.46000001  0.49000001]]
[[ 0.01        0.47999999  0.47999999  0.49000001  0.5         0.47        0.44
   0.46000001  0.46000001  0.49000001]]
[[ 0.01        0.47999999  0.47999999  0.47999999  0.5         0.47        0.44
   0.46000001  0.46000001  0.49000001]]
[[ 0.01        0.47999999  0.47        0.47999999  0.5         0.46000001
   0.44        0.44999999  0.46000001  0.49000001]]
[[ 0.01        0.46000001  0.46000001  0.47        0.47999999  0.44999999
   0.43000001  0.44        0.44        0.47999999]]
[[ 0.01        0.46000001  0.46000001  0.46000001  0.47999999  0.44999999
   0.41999999  0.43000001  0.44        0.47      ]]
[[ 0.01   

[[ 0.01        0.44        0.44        0.44999999  0.46000001  0.43000001
   0.40000001  0.41999999  0.41999999  0.44999999]]
[[ 0.01        0.44        0.44        0.44999999  0.46000001  0.43000001
   0.41        0.41999999  0.43000001  0.46000001]]
[[ 0.01        0.44999999  0.44999999  0.44999999  0.47        0.43000001
   0.41        0.41999999  0.43000001  0.46000001]]
[[ 0.01        0.44999999  0.44999999  0.46000001  0.47        0.44        0.41
   0.43000001  0.43000001  0.46000001]]
[[ 0.01        0.44999999  0.44        0.44999999  0.46000001  0.43000001
   0.41        0.41999999  0.43000001  0.46000001]]
[[ 0.01        0.49000001  0.49000001  0.5         0.50999999  0.47999999
   0.44999999  0.47        0.47        0.5       ]]
[[ 0.01        0.49000001  0.49000001  0.49000001  0.50999999  0.47999999
   0.44999999  0.46000001  0.47        0.5       ]]
[[ 0.01        0.49000001  0.49000001  0.5         0.50999999  0.47999999
   0.44999999  0.47        0.47        0.5       ]

[[ 0.01        0.44        0.44        0.44999999  0.46000001  0.43000001
   0.41        0.41999999  0.41999999  0.44999999]]
[[ 0.01        0.49000001  0.49000001  0.49000001  0.50999999  0.47999999
   0.44999999  0.47        0.47        0.5       ]]
[[ 0.01        0.5         0.5         0.5         0.51999998  0.49000001
   0.46000001  0.47        0.47999999  0.50999999]]
[[ 0.01        0.5         0.5         0.50999999  0.51999998  0.49000001
   0.47        0.47999999  0.49000001  0.51999998]]
[[ 0.01        0.50999999  0.5         0.50999999  0.52999997  0.49000001
   0.47        0.47999999  0.49000001  0.51999998]]
[[ 0.01        0.50999999  0.50999999  0.51999998  0.52999997  0.5         0.47
   0.49000001  0.49000001  0.51999998]]
[[ 0.01        0.50999999  0.50999999  0.51999998  0.52999997  0.5         0.47
   0.49000001  0.49000001  0.51999998]]
[[ 0.01        0.51999998  0.50999999  0.51999998  0.54000002  0.5
   0.47999999  0.49000001  0.5         0.52999997]]
[[ 0.01    

In [14]:
arquivo

,Hora,Temperature,Wind_Speed,Wind_Direction,Humidity,Air_Density,Rain_Status,Jardim_PMV,Jardim_Temp,Sala_PMV,...,PortaSalaNorte,JanelaSalaNorte,PortaSalaLeste,JanelaSalaSul,JanelaSalaOeste,PortaCozinhaLeste,JanelaCozinhaSul,JanelaMezaninoNorte,JanelaMezaninoSul,JanelaMezaninoOeste
